## Dans cette séance, vous devez créer un nouveau chatbot, ayant certaines caractéristiques de Parry.
#### Travail à faire : Pour la création de Parry, vous pouvez vous baser sur le code utilisé pour programmer Eliza et l'adapter sur les points suivants :

1. Ajouter les trois variables affectives (peur, colère et méfiance),
2. Avant de traiter la phrase tapée par l'utilisateur, vous devrez l'analyser et modifier les valeurs des trois variables affectives, en conséquence.
3. Vous devrez mettre au point la fonctionalité de sélection de la stratégie de réponse en vous inspirant de l'architecture de Parry vue en cours,
4. Modifier la syntaxe du script (doctor.txt) pour permettre d'asscocier chaque règle de transformation (reasmb) à une ou plusieurs stratégies : la règle sera alors sélectionnée seulement si elle est associée à la stratégie courante (ex. si la stratégie courante est peur, seules les reasmb associées à peur pourront être sélectionnées).
5. Modifier le code du chatbot de façon à ce qu'il soit adapté à cette nouvelle syntaxe du script.


### Variables d'influences 

* **Peur et colère (de 0 à 20)**
* **Méfance (de 0 à 15)**

**Conditions initiales : Toutes faibles**
   * Après chaque réplique de l'utilisateur, si rien de malveillant dans la saisie
       * – La colère diminue de 1, la peur de 0,3
       * – La méfance baisse de 0,05 par rapport au niveau de base
   * Sinon, en cas de malveillance, cela dépend de ce que dit l'utilisateur
       * – Chaque déclaration de l’utilisateur peut changer la peur et la colère
           * • Les insultes augmentent la colère d'un certain pourcentage
       * – La méfance augmente en même temps que la peur ou la colère


### Compléxité des règles entrée/sortie

* L'utilisateur insinue que Parry est un malade mental
  *  La montée de la peur et de la colère
* L'utilisateur mentionne "Mafia" ou des concepts associés ("tuer") :
  *  montée de la peur 
  *  la volonté de discuter dépend des niveaux actuels de Peur, de Colère, de Méfance
* L'utilisateur mentionne Parry
    * Flatterie (réaction positive) 
      * Diminution de la peur/du danger si la méfance est faible
      * Augmente la colère si la méfance est élevée
  * Attitude des utilisateurs à l'égard de Parry
      * Les attitudes négatives (peur, incrédulité) augmentent la peur/la colère


### Sujets sensibles (par ordre)

**chevaux → courses de chevaux → jeux d’hasard →bookmakers→ police → mafia**
* Le fait que l'interviewer mentione un nouveau sujet sensible provoque une augmentation de la peur
* Les sujets sensibles suscitent des réponses pré-affectées de la part de Parry


In [31]:
#pip install text2emotion
#pip install profanity_filter

#Nos sujets sensibles
SujetSensible = ["horses","horses race","gambling","bookmakers","police","mafia"]

#Une librairie qui met des étoiles sur les "profanity", ça nous permet d'identifier les insults
from profanity_filter import ProfanityFilter
pf = ProfanityFilter(languages=['en'])
pf.censor("That's bullshit!")
#"That's ********!"

#to get emotion in sentence
import text2emotion as te
te.get_emotion("Policeman don't do their job!")#The output we received,
#{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.5, 'Fear': 0.5}



{'Happy': 0.0, 'Angry': 0.0, 'Surprise': 0.0, 'Sad': 0.5, 'Fear': 0.5}

In [1]:
# Fix Python2/Python3 incompatibility
try: input = raw_input
except NameError: pass

log = logging.getLogger(__name__)


class Key:
    def __init__(self, word, weight, decomps):
        self.word = word
        self.weight = weight
        self.decomps = decomps

class Decomp:
    def __init__(self, parts, save, reasmbs):
        self.parts = parts
        self.save = save
        self.reasmbs = reasmbs
        self.next_reasmb_index = 0


class Eliza:
    def __init__(self):
        self.initials = []
        self.finals = []
        self.quits = []
        self.pres = {}
        self.posts = {}
        self.synons = {}
        self.keys = {}
        self.memory = []
        
        self.peur = 5
        self.colere = 5
        self.mefiance = 5

    def load(self, path):
        key = None
        decomp = None
        with open(path) as file:
            for line in file:
                if not line.strip():
                    continue
                tag, content = [part.strip() for part in line.split(':')]
                if tag == 'initial':
                    self.initials.append(content)
                elif tag == 'final':
                    self.finals.append(content)
                elif tag == 'quit':
                    self.quits.append(content)
                elif tag == 'pre':
                    parts = content.split(' ')
                    self.pres[parts[0]] = parts[1:]
                elif tag == 'post':
                    parts = content.split(' ')
                    self.posts[parts[0]] = parts[1:]
                elif tag == 'synon':
                    parts = content.split(' ')
                    self.synons[parts[0]] = parts
                elif tag == 'key':
                    parts = content.split(' ')
                    word = parts[0]
                    weight = int(parts[1]) if len(parts) > 1 else 1
                    key = Key(word, weight, [])
                    self.keys[word] = key
                elif tag == 'decomp':
                    parts = content.split(' ')
                    save = False
                    if parts[0] == '$':
                        save = True
                        parts = parts[1:]
                    decomp = Decomp(parts, save, [])
                    key.decomps.append(decomp)
                elif tag == 'reasmb':
                    parts = content.split(' ')
                    decomp.reasmbs.append(parts)

    def _match_decomp_r(self, parts, words, results):
        if not parts and not words:
            return True
        if not parts or (not words and parts != ['*']):
            return False
        if parts[0] == '*':
            for index in range(len(words), -1, -1):
                results.append(words[:index])
                if self._match_decomp_r(parts[1:], words[index:], results):
                    return True
                results.pop()
            return False
        elif parts[0].startswith('@'):
            root = parts[0][1:]
            if not root in self.synons:
                raise ValueError("Unknown synonym root {}".format(root))
            if not words[0].lower() in self.synons[root]:
                return False
            results.append([words[0]])
            return self._match_decomp_r(parts[1:], words[1:], results)
        elif parts[0].lower() != words[0].lower():
            return False
        else:
            return self._match_decomp_r(parts[1:], words[1:], results)

    def _match_decomp(self, parts, words):
        results = []
        if self._match_decomp_r(parts, words, results):
            return results
        return None

    
    def _next_reasmb(self, decomp):
        #on prend l'index de la reponse qui au début est 0
        index = decomp.next_reasmb_index
        #on utilise cet index pour récupérer la réponse dans decomp de la class Key
        result = decomp.reasmbs[index % len(decomp.reasmbs)]
        
        #j'ajoute un random choice de l'index pour avoir une réponse aléatoire à partir de la décomposition de réponse dispo
        #Lorsque l'index choisi est différent du précédent, alors on l'utilise pour la prochaine réponse provenant
        #De la meme décomposition
        #Cependant si l'index est le meme que précédement, alors on lui ajoute +1 (ce qui été déjà le cas initialement)
        n=random.choice(range(len(decomp.reasmbs)))
        if n!= index:
            decomp.next_reasmb_index = n
        else :
            decomp.next_reasmb_index = index + 1
            
        return result

    
    def _reassemble(self, reasmb, results):
        output = []
        for reword in reasmb:
            if not reword:
                continue
            if reword[0] == '(' and reword[-1] == ')':
                index = int(reword[1:-1])
                if index < 1 or index > len(results):
                    raise ValueError("Invalid result index {}".format(index))
                insert = results[index - 1]
                for punct in [',', '.', ';']:
                    if punct in insert:
                        insert = insert[:insert.index(punct)]
                output.extend(insert)
            else:
                output.append(reword)
        return output

    def _sub(self, words, sub):
        output = []
        for word in words:
            word_lower = word.lower()
            if word_lower in sub:
                output.extend(sub[word_lower])
            else:
                output.append(word)
        return output

    def _match_key(self, words, key):
        for decomp in key.decomps:
            results = self._match_decomp(decomp.parts, words)
            if results is None:
                log.debug('Decomp did not match: %s', decomp.parts)
                continue
            log.debug('Decomp matched: %s', decomp.parts)
            log.debug('Decomp results: %s', results)
            results = [self._sub(words, self.posts) for words in results]
            log.debug('Decomp results after posts: %s', results)
            reasmb = self._next_reasmb(decomp)
            log.debug('Using reassembly: %s', reasmb)
            if reasmb[0] == 'goto':
                goto_key = reasmb[1]
                if not goto_key in self.keys:
                    raise ValueError("Invalid goto key {}".format(goto_key))
                log.debug('Goto key: %s', goto_key)
                return self._match_key(words, self.keys[goto_key])
            output = self._reassemble(reasmb, results)
            if decomp.save:
                self.memory.append(output)
                log.debug('Saved to memory: %s', output)
                continue
            return output
        return None

    def respond(self, text):
        
        if text.lower() in self.quits:
            return None

        text = re.sub(r'\s*\.+\s*', ' . ', text)
        text = re.sub(r'\s*,+\s*', ' , ', text)
        text = re.sub(r'\s*;+\s*', ' ; ', text)
        
        #on modifie "?" en " ? "
        text = re.sub(r'\s*\?+\s*', ' ? ', text)
        
        #Pour faire pour tous les charactères spéciaux etc..
        text = ''.join([e for e in text if e.isalnum() or e.isspace()])

        log.debug('After punctuation cleanup: %s', text)

        words = [w for w in text.split(' ') if w]
        log.debug('Input: %s', words)

        words = self._sub(words, self.pres)
        log.debug('After pre-substitution: %s', words)

        keys = [self.keys[w.lower()] for w in words if w.lower() in self.keys]
        keys = sorted(keys, key=lambda k: -k.weight)
        log.debug('Sorted keys: %s', [(k.word, k.weight) for k in keys])

        output = None

        for key in keys:
            output = self._match_key(words, key)
            if output:
                log.debug('Output from key: %s', output)
                break
        if not output:
            if self.memory:
                index = random.randrange(len(self.memory))
                output = self.memory.pop(index)
                log.debug('Output from memory: %s', output)
            else:
                output = self._next_reasmb(self.keys['xnone'].decomps[0])
                log.debug('Output from xnone: %s', output)

        return " ".join(output)

    def initial(self):
        return random.choice(self.initials)

    def final(self):
        return random.choice(self.finals)
    

    def run(self):
        print(self.initial())
        input_text=["last","actual"]
        
        while True:
            sent = input('> ')
            
            
            
            
            input_text.append(sent)
            if levCalculate(input_text[-1], input_text[-2]) > 0.7 :
                output= "Stop repeting yourself !"
            else:
                output = self.respond(sent)
                if output is None:
                    break
            print(output)

        print(self.final())


        
# Ma fonction de calcul de distance de Levenstein
def levCalculate(str1, str2):
    Ratio = lev.ratio(str1, str2)
    return Ratio


def main():
    eliza = Eliza()
    eliza.load('doctor.txt')
    eliza.run()

if __name__ == '__main__':
    logging.basicConfig()
    main()


How do you do.  Please tell me your problem.
> What is your name?
I am not interested in names.
> What is your name ? 
I've told you before, I don't care about names -- please continue.
> bye
Goodbye.  Thank you for talking to me.


In [12]:
message = ': Learn Python '

# remove leading and trailing ":"
print('Message', message.strip(":"))

Message  Learn Python 


In [17]:
a=["maybe", "perhaps"]

    
print(a[0])
print(a[1:])

maybe
['perhaps']
